In [1]:
import gzip
#mitosheet.sheet()
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
variant_disease = pd.read_csv("all_variant_disease_associations.tsv.gz", compression='gzip', sep = '\t')

In [3]:
print(np.shape(np.unique(variant_disease['diseaseName'])))
np.unique(variant_disease['diseaseName'])

(14155,)


array(['1-3 toe syndactyly', '1-5 finger syndactyly', '11 pairs of ribs',
       ..., 'von Willebrand disease type 2M',
       "von Willebrand's factor (lab test)", 'winter depression'],
      dtype=object)

In [4]:
Lung = list()
for i in range(len(variant_disease['diseaseName'])):
    if 'Lung' in variant_disease['diseaseName'][i] and 'Acute Lung Injury' not in variant_disease['diseaseName'][i] \
    and 'Infectious Lung Disorder' not in variant_disease['diseaseName'][i] \
    and 'Lung Diseases, Interstitial' not in variant_disease['diseaseName'][i] \
    and 'Lung Diseases, Obstructive' not in variant_disease['diseaseName'][i] \
    and 'Lung diseases' not in variant_disease['diseaseName'][i] \
    and 'Sex Reversal, Female, With Dysgenesis Of Kidneys, Adrenals, And Lungs' not in variant_disease['diseaseName'][i] \
    and 'Transfusion-Related Acute Lung Injury' not in variant_disease['diseaseName'][i]:
        Lung.append(i)
    if 'lung' in variant_disease['diseaseName'][i] \
    and 'Bulla of lung' not in variant_disease['diseaseName'][i] \
    and 'Chronic lung disease' not in variant_disease['diseaseName'][i] \
    and 'Congenital hypoplasia of lung' not in variant_disease['diseaseName'][i] \
    and 'Madelung Deformity' not in variant_disease['diseaseName'][i] :
        Lung.append(i)
print(len(np.unique(variant_disease['diseaseName'][Lung])))
np.unique(variant_disease['diseaseName'][Lung])

32


array(['Adenocarcinoma of lung (disorder)',
       'Adenocarcinoma of lung stage IIIB',
       'Adenocarcinoma of lung, stage I',
       'Adenocarcinoma of lung, stage IV', 'Adenoma of lung',
       'Adenosquamous cell lung cancer',
       'Advanced Lung Non-Squamous Non-Small Cell Carcinoma',
       'Advanced lung cancer', 'Benign neoplasm of lung',
       'Carcinoma of lung',
       'Epidermal growth factor receptor positive non-small cell lung cancer',
       'Large cell carcinoma of lung', 'Lung Neoplasms',
       'Malignant neoplasm of lung', 'Metastatic Lung Adenocarcinoma',
       'Metastatic non-small cell lung cancer',
       'Neuroendocrine neoplasm of lung', 'Non-Small Cell Lung Carcinoma',
       'Non-small cell lung cancer recurrent',
       'Non-small cell lung cancer stage I',
       'Non-small cell lung cancer stage III',
       'Non-squamous non-small cell lung cancer',
       'Primary malignant neoplasm of lung',
       'Secondary malignant neoplasm of lung',
       '

In [5]:
#These are information about the diseases from the database
# variant_disease.loc[Colon]

In [6]:
#Map the database to the geneSymbol names
snpId = variant_disease.loc[Lung]['snpId']
variant_gene = pd.read_csv("variant_to_gene_mappings.tsv.gz", compression='gzip', sep = '\t')
chosen = list()
for i in range(len(variant_gene['snpId'])):
    if variant_gene['snpId'][i] in np.array(snpId):
        chosen.append(i)
variant_gene.loc[chosen]

# Here is the list of GWAS genes that we are interested in
GWAS_Lung = np.unique(variant_gene.loc[chosen]['geneSymbol'])
print(len(GWAS_Lung))

1166


In [7]:
'''GWAS enrichment'''
'''DEGs are selected DSs'''
    
#DEGs of each celltype 
cancer_list = ['Lung','Liver','Breast','Colon','Ovary']
cancer = "Lung" # change here for different cancers

with open('/GWAS_enrich_percelltype_'+cancer+'.txt','w') as f:

    DEGs = pd.read_csv('/ligand_target_merged_withFC.csv',sep = ',')
    DEGs = DEGs.loc[DEGs['cancer']==cancer]
    DEGs

    celltype_DEG = np.unique(DEGs['receiver'])
    print("celltypes_DEGs: ", celltype_DEG, file =f)

    '''all_genes means the combine of all background genes in all celltypes'''
    bg_file_path = ("/background_genes_"+cancer+".csv")
    all_gene = np.arange(0)
    all = pd.read_csv(bg_file_path,sep = ',') 
    all = all.drop('Unnamed: 0',axis = 1)

    celltype_bg = all.columns
    print("celltypes_bgs: ", celltype_bg, file =f)
    DEGs

    '''for Lung only - loop all celltype'''
    for c in range(len(celltype_DEG)):
        print('\n', file =f)
        print(celltype_DEG[c], file =f)
        iDEGs = DEGs.loc[DEGs['receiver']==celltype_DEG[c]]['target'].unique()
        print(celltype_DEG[c],"DEGs",len(iDEGs), file =f)
        ibackground_genes = all[celltype_DEG[c]].unique()
        ibackground_genes = [x for x in ibackground_genes if x != 'NAN']
        ibackground_genes = [x for x in ibackground_genes if x != 'nan']
        print(celltype_DEG[c],"background",len(ibackground_genes), file =f)

        GWAS = list()
        for g in GWAS_Lung:
            for a in ibackground_genes:
                a = str(a)
                for i in range(len(a.split('///'))):
                    if g == a.split('///')[i]:
                        GWAS.append(a)                   
        GWAS = np.unique(GWAS)
        GWAS

        nonDEGs = list(set(ibackground_genes) - set(iDEGs))                
        table11 = len(np.intersect1d(GWAS, iDEGs))
        table12 = len(iDEGs) - table11
        table21 = len(np.intersect1d(GWAS, nonDEGs))
        table22 = len(ibackground_genes) - (table21 + table12 + table11)
        table = pd.DataFrame([[table11, table12], [table21, table22]])
        table.index = ('DEGs', 'nonDEGs')
        table.columns = ('GWAS', 'nonGWAS')

        print('Lung Cancer - ',celltype_DEG[c], file =f )
        print(table, file =f)
        odds, pval = stats.fisher_exact(table)
        print("Odds ratio: ", round(odds, 4), file =f)
        print("p-value: ",round(pval, 6), file =f)

        # if the odds are > 1 and pval <0.05 The list of DEGs is significantly enriched by the GWAS genes
        # if the odds are < 1 and pval <0.05 The list of DEGs is not enriched by the GWAS genes


In [8]:
'''GWAS enrichment'''
'''DEGs are selected all DSs
    background genes are all differentially expressed genes
'''
    
#DEGs of each celltype 
cancer_list = ['Lung','Liver','Breast','Colon','Ovary']
cancer = "Lung" # change here for different cancers

with open('/GWAS_enrich_percelltype_DEGasBG_'+cancer+'.txt','w') as f:

    DEGs = pd.read_csv('/ligand_target_merged_withFC.csv',sep = ',')
    DEGs = DEGs.loc[DEGs['cancer']==cancer]
    DEGs

    celltype_DEG = np.unique(DEGs['receiver'])
    print("celltypes_DEGs: ", celltype_DEG, file =f)

    '''Differentially expressed genes in each cell type were background genes'''
    bg_file_path = ("/DEGs_of_"+cancer+"/DEGs_merged_"+cancer+"_MAST.txt")
    all_gene = np.arange(0)
    all = pd.read_csv(bg_file_path,sep = '\t') 
    all
    
    celltype_bg = all.columns
    print("celltypes_bgs: ", celltype_bg, file =f)
    DEGs

    '''loop all celltype'''
    for c in range(len(celltype_DEG)):
        print('\n', file =f)
        print(celltype_DEG[c], file =f)
        iDEGs = DEGs.loc[DEGs['receiver']==celltype_DEG[c]]['target'].unique()
        print(celltype_DEG[c],"DEGs",len(iDEGs), file =f)
        ibackground_genes = all[celltype_DEG[c]].unique()
        ibackground_genes = [x for x in ibackground_genes if x != 'NAN']
        ibackground_genes = [x for x in ibackground_genes if x != 'nan']
        print(celltype_DEG[c],"background",len(ibackground_genes), file =f)

        GWAS = list()
        for g in GWAS_Lung:
            for a in ibackground_genes:
                a = str(a)
                for i in range(len(a.split('///'))):
                    if g == a.split('///')[i]:
                        GWAS.append(a)                   
        GWAS = np.unique(GWAS)
        GWAS

        nonDEGs = list(set(ibackground_genes) - set(iDEGs))                
        table11 = len(np.intersect1d(GWAS, iDEGs))
        table12 = len(iDEGs) - table11
        table21 = len(np.intersect1d(GWAS, nonDEGs))
        table22 = len(ibackground_genes) - (table21 + table12 + table11)
        table = pd.DataFrame([[table11, table12], [table21, table22]])
        table.index = ('DEGs', 'nonDEGs')
        table.columns = ('GWAS', 'nonGWAS')

        print('Lung Cancer - ',celltype_DEG[c], file =f )
        print(table, file =f)
        odds, pval = stats.fisher_exact(table)
        print("Odds ratio: ", round(odds, 4), file =f)
        print("p-value: ",round(pval, 6), file =f)

        # if the odds are > 1 and pval <0.05 The list of DEGs is significantly enriched by the GWAS genes
        # if the odds are < 1 and pval <0.05 The list of DEGs is not enriched by the GWAS genes



In [11]:
'''GWAS enrichment'''
'''DEGs are selected all DSs of the scURs
    background genes are all differentially expressed genes
'''
    
#DEGs of each celltype 
cancer_list = ['Lung','Liver','Breast','Colon','Ovary']
cancer = "Lung" # change here for different cancers

with open('/GWAS_enrich_percelltype_DS4scUR_DEGasBG_'+cancer+'.txt','w') as f:

    DEGs = pd.read_csv('/ligand_target_merged_DS_of_scUR.csv',sep = ',')
    #top_scURs = ["COL1A1","FN1","SPP1","COL4A1","COL18A1", "PLAU" , "CLEC11A" ,"MDK"]
    #DEGs = DEGs.loc[np.isin(DEGs['ligand'], top_scURs)]
    DEGs = DEGs.loc[DEGs['cancer']==cancer]


    celltype_DEG = np.unique(DEGs['receiver'])
    print("celltypes_DEGs: ", celltype_DEG, file =f)

    '''Differentially expressed genes in each cell type were background genes'''
    bg_file_path = ("/DEGs_of_"+cancer+"/DEGs_merged_"+cancer+"_MAST.txt")
    all_gene = np.arange(0)
    all = pd.read_csv(bg_file_path,sep = '\t') 
    all
    
    celltype_bg = all.columns
    print("celltypes_bgs: ", celltype_bg, file =f)
    DEGs

    '''loop all celltype'''
    for c in range(len(celltype_DEG)):
        print('\n', file =f)
        print(celltype_DEG[c], file =f)
        iDEGs = DEGs.loc[DEGs['receiver']==celltype_DEG[c]]['target'].unique()
        print(celltype_DEG[c],"DEGs",len(iDEGs), file =f)
        ibackground_genes = all[celltype_DEG[c]].unique()
        ibackground_genes = [x for x in ibackground_genes if x != 'NAN']
        ibackground_genes = [x for x in ibackground_genes if x != 'nan']
        print(celltype_DEG[c],"background",len(ibackground_genes), file =f)

        GWAS = list()
        for g in GWAS_Lung:
            for a in ibackground_genes:
                a = str(a)
                for i in range(len(a.split('///'))):
                    if g == a.split('///')[i]:
                        GWAS.append(a)                   
        GWAS = np.unique(GWAS)
        GWAS

        nonDEGs = list(set(ibackground_genes) - set(iDEGs))                
        table11 = len(np.intersect1d(GWAS, iDEGs))
        table12 = len(iDEGs) - table11
        table21 = len(np.intersect1d(GWAS, nonDEGs))
        table22 = len(ibackground_genes) - (table21 + table12 + table11)
        table = pd.DataFrame([[table11, table12], [table21, table22]])
        table.index = ('DEGs', 'nonDEGs')
        table.columns = ('GWAS', 'nonGWAS')

        print('Lung Cancer - ',celltype_DEG[c], file =f )
        print(table, file =f)
        odds, pval = stats.fisher_exact(table)
        print("Odds ratio: ", round(odds, 4), file =f)
        print("p-value: ",round(pval, 6), file =f)

        # if the odds are > 1 and pval <0.05 The list of DEGs is significantly enriched by the GWAS genes
        # if the odds are < 1 and pval <0.05 The list of DEGs is not enriched by the GWAS genes




In [10]:
# list GWAS gene that in our gene-of-interests
# np.intersect1d(GWAS, DEGs)

array(['BAX', 'CCND1', 'GADD45B', 'GNAS', 'KLF6', 'KRT8', 'MMP2', 'MMP9',
       'MUC1', 'NR3C1', 'PLAU', 'SPP1', 'TGFB1', 'TIMP1'], dtype=object)